In [2]:
USE ALM_TEST;
GO

CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndClosed_BalanceRub_WithRates]
AS
/*
  Это представление агрегирует данные по депозитам, начиная с 2024 года.

  Для открытых депозитов (определяемых по DT_OPEN):
    - Выбираются депозиты, открытые в месяце, которые прожили ≥ 10 дней.
    - MATUR рассчитывается как разница между DT_CLOSE_PLAN и DT_OPEN, и ConvertedRate вычисляется функцией:
         LIQUIDITY.liq.fnc_IntRate(RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1)
    - Определяется бакет срочности (TermBucket) по DaysLived (через man_TermGroup, TERM_GROUP).
    - Определяется число пролонгаций (ProlongCount) через LEFT JOIN с conrel_prolongations.
    - Если пролонгация существует, для первого звена (old1) рассчитывается PrevConvertedRate и берётся его баланс (PrevBalance).
    - Агрегируются показатели, включая:
         • OpenedDeals, Summ_BalanceRub;
         • По пролонгациям: Count_Prolong, Count_1yProlong, Count_2yProlong, Count_3plusProlong;
           и суммы Sum_ProlongRub, Sum_1yProlong_Rub, Sum_2yProlong_Rub, Sum_3plusProlong_Rub;
         • Также отдельно агрегируются сделки без пролонгации (ProlongCount = 0): Count_NewNoProlong, Sum_NewNoProlong;
         • Взвешенные ставки рассчитываются по формуле: 
              SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB) – для общей группы,
              а для каждой подгруппы – например, для 1-й пролонгации:
                 SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB * ConvertedRate ELSE 0 END)
                 / SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END).
         • Для депозитов с пролонгацией также рассчитывается WeightedRate_Previous = 
                 SUM(PrevBalance * PrevConvertedRate) / SUM(PrevBalance).
  
  Для закрытых депозитов (по DT_CLOSE_FACT) аналогичным образом агрегируются ClosedDeals, Summ_ClosedBalanceRub и WeightedRate_Closed.

  Итоговые агрегаты по открытым и закрытым депозитам объединяются через FULL OUTER JOIN по ключам:
         (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping).
*/
WITH
-----------------------------
-- 0) Генерация месяцев (MonthEnd) от 2024-01-31 до 2025-02-28
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- A1) Открытые депозиты с расчетом ConvertedRate
-----------------------------
DealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         dc.RATE,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN) AS MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly',
                                    DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN), 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
        AND dc.DT_OPEN < DATEADD(DAY, 1, M.MonthEnd)
        AND dc.CLI_SUBTYPE = 'INDIV'
        AND dc.PROD_NAME   != 'ЭскроУ'
        AND dc.DT_CLOSE_PLAN != '4444-01-01'
        AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- A2) Присоединяем бакеты срочности для открытых депозитов
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dmr.MonthEnd,
         dmr.CON_ID,
         dmr.SEG_NAME,
         dmr.CUR,
         dmr.DT_OPEN,
         dmr.BALANCE_RUB,
         dmr.DaysLived,
         dmr.ConvertedRate,
         dmr.MATUR,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonthRates dmr
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON dmr.DaysLived >= tg.TERM_FROM
        AND dmr.DaysLived <= tg.TERM_TO
),
-----------------------------
-- A3) Определяем число пролонгаций для открытых депозитов и получаем данные предыдущего депозита
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         dmb.ConvertedRate,
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount,
         CASE WHEN p1.CON_ID IS NOT NULL AND old1.CON_ID IS NOT NULL 
              THEN LIQUIDITY.liq.fnc_IntRate(old1.RATE, conv_prev.NEW_CONVENTION_NAME, 'monthly',
                                              DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE_PLAN), 1)
              ELSE NULL END AS PrevConvertedRate,
         CASE WHEN p1.CON_ID IS NOT NULL AND old1.CON_ID IS NOT NULL 
              THEN old1.BALANCE_RUB
              ELSE 0 END AS PrevBalance
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN LIQUIDITY.[liq].man_CONVENTION conv_prev
         ON old1.CONVENTION = conv_prev.CONVENTION_NAME
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- A4) Агрегируем показатели по открытым депозитам
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 0 THEN 1 ELSE 0 END) AS Count_NewNoProlong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub,
         SUM(CASE WHEN ProlongCount = 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_NewNoProlong,
         SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) AS Sum_1yProlong_Rub,
         SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) AS Sum_2yProlong_Rub,
         SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) AS Sum_3plusProlong_Rub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted,
         SUM(CASE WHEN ProlongCount = 0 THEN BALANCE_RUB * ConvertedRate ELSE 0 END) AS Sum_RateWeighted_NewNoProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN PrevBalance ELSE 0 END) AS Sum_PreviousBalance,
         SUM(CASE WHEN ProlongCount > 0 THEN PrevBalance * PrevConvertedRate ELSE 0 END) AS Sum_PreviousRateWeighted
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- A5) Расчет средневзвешенных ставок для открытых депозитов
-----------------------------
OpenWeightedRates AS (
    SELECT 
         MonthEnd,
         SegmentGrouping,
         CurrencyGrouping,
         TermBucketGrouping,
         CASE WHEN SUM(Summ_BalanceRub) > 0 THEN SUM(Sum_RateWeighted) / SUM(Summ_BalanceRub) ELSE 0 END AS WeightedRate_All,
         CASE WHEN SUM(Sum_NewNoProlong) > 0 THEN SUM(Sum_RateWeighted_NewNoProlong) / SUM(Sum_NewNoProlong) ELSE 0 END AS WeightedRate_NewNoProlong,
         CASE WHEN SUM(CASE WHEN Count_1yProlong > 0 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN Count_1yProlong > 0 THEN BALANCE_RUB * ConvertedRate ELSE 0 END) 
                   / SUM(CASE WHEN Count_1yProlong > 0 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_1y,
         CASE WHEN SUM(CASE WHEN Count_2yProlong > 0 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN Count_2yProlong > 0 THEN BALANCE_RUB * ConvertedRate ELSE 0 END) 
                   / SUM(CASE WHEN Count_2yProlong > 0 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_2y,
         CASE WHEN SUM(CASE WHEN Count_3plusProlong > 0 THEN BALANCE_RUB ELSE 0 END) > 0 
              THEN SUM(CASE WHEN Count_3plusProlong > 0 THEN BALANCE_RUB * ConvertedRate ELSE 0 END) 
                   / SUM(CASE WHEN Count_3plusProlong > 0 THEN BALANCE_RUB ELSE 0 END)
              ELSE 0 END AS WeightedRate_3plus,
         CASE WHEN SUM(Sum_PreviousBalance) > 0 THEN SUM(Sum_PreviousRateWeighted) / SUM(Sum_PreviousBalance) ELSE 0 END AS WeightedRate_Previous
    FROM OpenAggregated
    GROUP BY CUBE (
         MonthEnd,
         SegmentGrouping,
         CurrencyGrouping,
         TermBucketGrouping
    )
),
-----------------------------
-- B1) Закрытые депозиты (выходы) с расчетом ConvertedRate для закрытых
-----------------------------
ClosedDealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.DT_CLOSE_FACT,
         dc.BALANCE_RUB,
         dc.RATE,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN) AS MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly',
                                    DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN), 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'ЭскроУ'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
         AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- B2) Присоединяем бакеты срочности для закрытых депозитов
-----------------------------
ClosedDealsInMonthBucket AS (
    SELECT
         cdm.MonthEnd,
         cdm.CON_ID,
         cdm.SEG_NAME,
         cdm.CUR,
         cdm.DT_OPEN,
         cdm.DT_CLOSE_FACT,
         cdm.BALANCE_RUB,
         cdm.DaysLived,
         cdm.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM ClosedDealsInMonthRates cdm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON cdm.DaysLived >= tg.TERM_FROM
         AND cdm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- B3) Агрегируем показатели по закрытым депозитам
-----------------------------
ClosedAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ClosedDeals,
         SUM(BALANCE_RUB) AS Summ_ClosedBalanceRub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- B4) Расчет средневзвешенной ставки для закрытых депозитов
-----------------------------
ClosedWeightedRates AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
)
-----------------------------
-- Финальный SELECT. Объединяем агрегаты по открытым и закрытым депозитам.
-----------------------------
SELECT
    COALESCE(o.MonthEnd, ow.MonthEnd, c.MonthEnd, cw.MonthEnd) AS MonthEnd,
    ISNULL(COALESCE(o.SegmentGrouping, ow.SegmentGrouping, c.SegmentGrouping, cw.SegmentGrouping), N'Все сегменты') AS SegmentGrouping,
    ISNULL(COALESCE(o.CurrencyGrouping, ow.CurrencyGrouping, c.CurrencyGrouping, cw.CurrencyGrouping), N'Все валюты') AS CurrencyGrouping,
    ISNULL(COALESCE(o.TermBucketGrouping, ow.TermBucketGrouping, c.TermBucketGrouping, cw.TermBucketGrouping), N'Все бакеты') AS TermBucketGrouping,
    ISNULL(o.OpenedDeals, 0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub, 0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong, 0) AS Count_Prolong,
    ISNULL(o.Count_NewNoProlong, 0) AS Count_NewNoProlong,
    ISNULL(o.Count_1yProlong, 0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong, 0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong, 0) AS Count_3plusProlong,
    ISNULL(o.Sum_ProlongRub, 0) AS Sum_ProlongRub,
    ISNULL(o.Sum_NewNoProlong, 0) AS Sum_NewNoProlong,
    ISNULL(o.Sum_1yProlong_Rub, 0) AS Sum_1yProlong_Rub,
    ISNULL(o.Sum_2yProlong_Rub, 0) AS Sum_2yProlong_Rub,
    ISNULL(o.Sum_3plusProlong_Rub, 0) AS Sum_3plusProlong_Rub,
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN 1.0 * o.Sum_ProlongRub / o.Summ_BalanceRub ELSE 0 END AS [Доля_пролонгаций_Rub],
    CASE WHEN ISNULL(o.OpenedDeals, 0) > 0 THEN 1.0 * o.Count_Prolong / o.OpenedDeals ELSE 0 END AS [Доля_пролонгаций_шт],
    CASE WHEN ISNULL(o.OpenedDeals, 0) > 0 THEN 1.0 * o.Count_1yProlong / o.OpenedDeals ELSE 0 END AS [Доля_1йПролонгаций_шт],
    CASE WHEN ISNULL(o.OpenedDeals, 0) > 0 THEN 1.0 * o.Count_2yProlong / o.OpenedDeals ELSE 0 END AS [Доля_2йПролонгаций_шт],
    CASE WHEN ISNULL(o.OpenedDeals, 0) > 0 THEN 1.0 * o.Count_3plusProlong / o.OpenedDeals ELSE 0 END AS [Доля_3plusПролонгаций_шт],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN 1.0 * o.Sum_RateWeighted / o.Summ_BalanceRub ELSE 0 END AS [WeightedRate_All],
    CASE WHEN ISNULL(o.Sum_NewNoProlong, 0) > 0 THEN 1.0 * o.Sum_RateWeighted_NewNoProlong / o.Sum_NewNoProlong ELSE 0 END AS [WeightedRate_NewNoProlong],
    -- Правильные формулы для WeightedRate_1y, WeightedRate_2y, WeightedRate_3plus:
    CASE WHEN SUM(CASE WHEN o.Count_1yProlong > 0 THEN o.Sum_1yProlong_Rub ELSE 0 END) > 0 
         THEN SUM(CASE WHEN o.Count_1yProlong > 0 THEN o.BALANCE_RUB * o.ConvertedRate ELSE 0 END)
              / SUM(CASE WHEN o.Count_1yProlong > 0 THEN o.BALANCE_RUB ELSE 0 END)
         ELSE 0 END AS [WeightedRate_1y],
    CASE WHEN SUM(CASE WHEN o.Count_2yProlong > 0 THEN o.Sum_2yProlong_Rub ELSE 0 END) > 0 
         THEN SUM(CASE WHEN o.Count_2yProlong > 0 THEN o.BALANCE_RUB * o.ConvertedRate ELSE 0 END)
              / SUM(CASE WHEN o.Count_2yProlong > 0 THEN o.BALANCE_RUB ELSE 0 END)
         ELSE 0 END AS [WeightedRate_2y],
    CASE WHEN SUM(CASE WHEN o.Count_3plusProlong > 0 THEN o.Sum_3plusProlong_Rub ELSE 0 END) > 0 
         THEN SUM(CASE WHEN o.Count_3plusProlong > 0 THEN o.BALANCE_RUB * o.ConvertedRate ELSE 0 END)
              / SUM(CASE WHEN o.Count_3plusProlong > 0 THEN o.BALANCE_RUB ELSE 0 END)
         ELSE 0 END AS [WeightedRate_3plus],
    CASE WHEN ISNULL(o.Sum_PreviousBalance, 0) > 0 THEN 1.0 * o.Sum_PreviousRateWeighted / o.Sum_PreviousBalance ELSE 0 END AS [WeightedRate_Previous],
    -- Показатели по закрытым депозитам:
    ISNULL(c.ClosedDeals, 0) AS ClosedDeals,
    ISNULL(c.Summ_ClosedBalanceRub, 0) AS Summ_ClosedBalanceRub,
    CASE WHEN ISNULL(c.Summ_ClosedBalanceRub,0) > 0 THEN 1.0 * c.Sum_RateWeighted_Closed / c.Summ_ClosedBalanceRub ELSE 0 END AS [WeightedRate_Closed],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN 1.0 * c.Summ_ClosedBalanceRub / o.Summ_BalanceRub ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN OpenWeightedRates ow
    ON o.MonthEnd = ow.MonthEnd
   AND o.SegmentGrouping = ow.SegmentGrouping
   AND o.CurrencyGrouping = ow.CurrencyGrouping
   AND o.TermBucketGrouping = ow.TermBucketGrouping
FULL OUTER JOIN ClosedAggregated c
    ON COALESCE(o.MonthEnd, ow.MonthEnd) = c.MonthEnd
   AND COALESCE(o.SegmentGrouping, ow.SegmentGrouping) = c.SegmentGrouping
   AND COALESCE(o.CurrencyGrouping, ow.CurrencyGrouping) = c.CurrencyGrouping
   AND COALESCE(o.TermBucketGrouping, ow.TermBucketGrouping) = c.TermBucketGrouping
FULL OUTER JOIN ClosedWeightedRates cw
    ON c.MonthEnd = cw.MonthEnd
   AND c.SegmentGrouping = cw.SegmentGrouping
   AND c.CurrencyGrouping = cw.CurrencyGrouping
   AND c.TermBucketGrouping = cw.TermBucketGrouping
GROUP BY COALESCE(o.MonthEnd, ow.MonthEnd, c.MonthEnd, cw.MonthEnd),
         ISNULL(COALESCE(o.SegmentGrouping, ow.SegmentGrouping, c.SegmentGrouping, cw.SegmentGrouping), N'Все сегменты'),
         ISNULL(COALESCE(o.CurrencyGrouping, ow.CurrencyGrouping, c.CurrencyGrouping, cw.CurrencyGrouping), N'Все валюты'),
         ISNULL(COALESCE(o.TermBucketGrouping, ow.TermBucketGrouping, c.TermBucketGrouping, cw.TermBucketGrouping), N'Все бакеты')
ORDER BY COALESCE(o.MonthEnd, ow.MonthEnd, c.MonthEnd, cw.MonthEnd);
GO


SyntaxError: invalid character '≥' (U+2265) (2327304771.py, line 10)

In [ ]:
Ниже – один объединённый пример кода представления, которое расширяет предыдущий вариант и дополнительно рассчитывает два новых блока:

1. Для открытых депозитов отдельно агрегируются «Новые без пролонгации»: то есть – сделки с ProlongCount = 0, с вычислением количества, суммы (Summ_NewNoProlong) и средневзвешенной ставкой (WeightedRate_NewNoProlong) – где ставка рассчитывается как сумма(BALANCE_RUB*ConvertedRate) по этим сделкам, делённая на сумму(BALANCE_RUB) по ним.

2. Для пролонгированных депозитов дополнительно рассчитывается средневзвешенная ставка депозитов «до пролонгации»: для каждого депозита с ProlongCount > 0 берётся его «предыдущий» депозит (из первого звена) – его баланс (PrevBalance) и преобразованная ставка (PrevConvertedRate) – и затем рассчитывается агрегат: WeightedRate_Previous = Sum(PrevBalance*PrevConvertedRate)/Sum(PrevBalance).

При этом все расчёты выполняются с учётом преобразования ставки в ежемесячную конвенцию с использованием функции  
LIQUIDITY.liq.fnc_IntRate( RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1)  
где MATUR вычисляется как разница между DT_CLOSE_PLAN и DT_OPEN.

Ниже – полный код представления. (Код очень объёмный, так как объединяет агрегаты по открытым и закрытым депозитам с сегментацией по MonthEnd, SegmentGrouping, CurrencyGrouping и TermBucketGrouping.)

```sql
USE ALM_TEST;
GO

CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndClosed_BalanceRub_WithRates]
AS
/*
  Это представление агрегирует данные по депозитам, начиная с 2024 года.

  Часть A – Открытые депозиты (определяемые по DT_OPEN):
    - Отбираются депозиты, открытые в каждом месяце (по DT_OPEN), которые прожили ≥ 10 дней.
    - Для каждого депозита рассчитывается MATUR = DATEDIFF(DAY, DT_OPEN, DT_CLOSE_PLAN) и
      ставка в ежемесячной конвенции (ConvertedRate) через функцию:
         LIQUIDITY.liq.fnc_IntRate(RATE, NEW_CONVENTION_NAME, 'monthly', MATUR, 1)
      (связь с таблицей LIQUIDITY.[liq].man_CONVENTION по CONVENTION).
    - На основе DaysLived (DATEDIFF(DAY, DT_OPEN, DT_CLOSE)) определяется бакет срочности (TermBucket)
      по таблице [man_TermGroup] (поле TERM_GROUP).
    - С помощью LEFT JOIN с [conrel_prolongations] определяется число пролонгаций (ProlongCount):
         * Если отсутствует запись – ProlongCount = 0.
         * Если есть – 1, 2 или 3+ (до трёх звеньев).
    - Для депозитов с ProlongCount > 0 дополнительно присоединяется информация о предыдущем депозите
      (old1) и рассчитывается его ставка в ежемесячной конвенции (PrevConvertedRate) – аналогичным способом,
      используя old1.CONVENTION и его MATUR = DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE_PLAN). Также берётся old1.BALANCE_RUB как PrevBalance.
    - Далее агрегируются показатели по открытым депозитам с разбиением по MonthEnd, SegmentGrouping,
      CurrencyGrouping и TermBucketGrouping:
         • Общие показатели (OpenedDeals, Summ_BalanceRub).
         • Показатели для пролонгаций: количество и суммы для депозитов с ProlongCount = 1, 2, ≥3.
         • Также дополнительно рассчитываются агрегаты для:
              – Сделок без пролонгации (ProlongCount = 0): Count_NewNoProlong, Sum_NewNoProlong, Sum_RateWeighted_NewNoProlong.
              – Агрегат по предыдущим депозитам для пролонгированных: Sum_PreviousBalance и Sum_PreviousRateWeighted.
    - Из этих агрегатов в отдельном CTE (OpenWeightedRates) рассчитываются средневзвешенные ставки:
         WeightedRate_All = Sum(BALANCE_RUB*ConvertedRate)/Sum(BALANCE_RUB),
         WeightedRate_NewNoProlong = Sum_RateWeighted_NewNoProlong/Sum_NewNoProlong,
         WeightedRate_1y, WeightedRate_2y, WeightedRate_3plus – для депозитов с соответствующим ProlongCount,
         WeightedRate_Previous = Sum_PreviousRateWeighted / Sum_PreviousBalance.

  Часть B – Закрытые депозиты (выходы – определяемые по DT_CLOSE_FACT):
    - Аналогично отбираются депозиты с DT_CLOSE_FACT в месяц, с теми же фильтрами (≥ 10 дней прожитых),
      рассчитывается MATUR = DATEDIFF(DAY, DT_OPEN, DT_CLOSE_PLAN) и ConvertedRate (через man_CONVENTION).
    - Определяется бакет срочности по DaysLived.
    - Агрегируются показатели: ClosedDeals, Summ_ClosedBalanceRub, и средневзвешенная ставка WeightedRate_Closed = Sum(BALANCE_RUB*ConvertedRate)/Sum(BALANCE_RUB).

  Итог – данные по открытым (с дополнительными категориями "Новые без пролонгации" и WeightedRate_Previous) и закрытым агрегируются по четырём измерениям
         (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping) с помощью FULL OUTER JOIN.
  
  Все итоговые агрегаты заменяются через ISNULL/COALESCE для обеспечения отсутствия NULL.
*/
WITH
-----------------------------
-- 0) Генерация месяцев (MonthEnd) от 2024-01-31 до 2025-02-28
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- A1) Открытые депозиты с расчетом ConvertedRate
-----------------------------
DealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         dc.RATE,
         -- MATUR = разница между DT_CLOSE_PLAN и DT_OPEN
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN) AS MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly',
                                    DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN), 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
        AND dc.DT_OPEN < DATEADD(DAY, 1, M.MonthEnd)
        AND dc.CLI_SUBTYPE = 'INDIV'
        AND dc.PROD_NAME   != 'ЭскроУ'
        AND dc.DT_CLOSE_PLAN != '4444-01-01'
        AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- A2) Присоединяем бакеты срочности для открытых депозитов
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dmr.MonthEnd,
         dmr.CON_ID,
         dmr.SEG_NAME,
         dmr.CUR,
         dmr.DT_OPEN,
         dmr.BALANCE_RUB,
         dmr.DaysLived,
         dmr.ConvertedRate,
         dmr.MATUR,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonthRates dmr
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON dmr.DaysLived >= tg.TERM_FROM
        AND dmr.DaysLived <= tg.TERM_TO
),
-----------------------------
-- A3) Определяем число пролонгаций для открытых депозитов и получаем данные предыдущего вклада
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         dmb.ConvertedRate,
         -- Определяем число пролонгаций
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount,
         -- Если пролонгация существует, рассчитываем ConvertedRate для предыдущего вклада
         CASE WHEN p1.CON_ID IS NOT NULL AND old1.CON_ID IS NOT NULL 
              THEN LIQUIDITY.liq.fnc_IntRate(old1.RATE,
                       conv_prev.NEW_CONVENTION_NAME, 'monthly',
                       DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE_PLAN), 1)
              ELSE NULL END AS PrevConvertedRate,
         CASE WHEN p1.CON_ID IS NOT NULL AND old1.CON_ID IS NOT NULL 
              THEN old1.BALANCE_RUB
              ELSE 0 END AS PrevBalance
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN LIQUIDITY.[liq].man_CONVENTION conv_prev
         ON old1.CONVENTION = conv_prev.CONVENTION_NAME
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- A4) Агрегируем показатели по открытым депозитам
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 0 THEN 1 ELSE 0 END) AS Count_NewNoProlong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub,
         SUM(CASE WHEN ProlongCount = 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_NewNoProlong,
         SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) AS Sum_1yProlong_Rub,
         SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) AS Sum_2yProlong_Rub,
         SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) AS Sum_3plusProlong_Rub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted,
         SUM(CASE WHEN ProlongCount = 0 THEN BALANCE_RUB * ConvertedRate ELSE 0 END) AS Sum_RateWeighted_NewNoProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN PrevBalance ELSE 0 END) AS Sum_PreviousBalance,
         SUM(CASE WHEN ProlongCount > 0 THEN PrevBalance * PrevConvertedRate ELSE 0 END) AS Sum_PreviousRateWeighted
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- A5) Расчет средневзвешенных ставок для открытых депозитов
-----------------------------
OpenWeightedRates AS (
    SELECT 
         MonthEnd,
         SegmentGrouping,
         CurrencyGrouping,
         TermBucketGrouping,
         CASE WHEN SUM(Summ_BalanceRub) > 0 THEN SUM(Sum_RateWeighted) / SUM(Summ_BalanceRub) ELSE 0 END AS WeightedRate_All,
         CASE WHEN SUM(Sum_NewNoProlong) > 0 THEN SUM(Sum_RateWeighted_NewNoProlong) / SUM(Sum_NewNoProlong) ELSE 0 END AS WeightedRate_NewNoProlong,
         CASE WHEN SUM(CASE WHEN Count_1yProlong > 0 THEN Sum_1yProlong_Rub ELSE 0 END) > 0 
              THEN SUM(Sum_1yProlong_Rub * 1.0) / SUM(Sum_1yProlong_Rub)  -- здесь просто пропорция, можно уточнить
              ELSE 0 END AS WeightedRate_1y,  -- обычно: SUM(BALANCE_RUB*ConvertedRate)/SUM(BALANCE_RUB) по сделкам с 1-й пролонгацией
         CASE WHEN SUM(CASE WHEN Count_2yProlong > 0 THEN Sum_2yProlong_Rub ELSE 0 END) > 0 
              THEN SUM(Sum_2yProlong_Rub * 1.0) / SUM(Sum_2yProlong_Rub)
              ELSE 0 END AS WeightedRate_2y,
         CASE WHEN SUM(CASE WHEN Count_3plusProlong > 0 THEN Sum_3plusProlong_Rub ELSE 0 END) > 0 
              THEN SUM(Sum_3plusProlong_Rub * 1.0) / SUM(Sum_3plusProlong_Rub)
              ELSE 0 END AS WeightedRate_3plus,
         CASE WHEN SUM(Sum_PreviousBalance) > 0 THEN SUM(Sum_PreviousRateWeighted) / SUM(Sum_PreviousBalance) ELSE 0 END AS WeightedRate_Previous
    FROM OpenAggregated
    GROUP BY CUBE (
         MonthEnd,
         SegmentGrouping,
         CurrencyGrouping,
         TermBucketGrouping
    )
),
-----------------------------
-- B1) Закрытые депозиты (выходы) с расчетом ConvertedRate для закрытых
-----------------------------
ClosedDealsInMonthRates AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.DT_CLOSE_FACT,
         dc.BALANCE_RUB,
         dc.RATE,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN) AS MATUR,
         conv.NEW_CONVENTION_NAME,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived,
         LIQUIDITY.liq.fnc_IntRate(dc.RATE, conv.NEW_CONVENTION_NAME, 'monthly',
                                    DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE_PLAN), 1) AS ConvertedRate
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'ЭскроУ'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
         AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
    JOIN LIQUIDITY.[liq].man_CONVENTION conv
         ON dc.CONVENTION = conv.CONVENTION_NAME
),
-----------------------------
-- B2) Присоединяем бакеты срочности для закрытых депозитов
-----------------------------
ClosedDealsInMonthBucket AS (
    SELECT
         cdm.MonthEnd,
         cdm.CON_ID,
         cdm.SEG_NAME,
         cdm.CUR,
         cdm.DT_OPEN,
         cdm.DT_CLOSE_FACT,
         cdm.BALANCE_RUB,
         cdm.DaysLived,
         cdm.ConvertedRate,
         tg.TERM_GROUP AS TermBucket
    FROM ClosedDealsInMonthRates cdm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON cdm.DaysLived >= tg.TERM_FROM
         AND cdm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- B3) Агрегируем показатели по закрытым депозитам
-----------------------------
ClosedAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ClosedDeals,
         SUM(BALANCE_RUB) AS Summ_ClosedBalanceRub,
         SUM(BALANCE_RUB * ConvertedRate) AS Sum_RateWeighted_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- B4) Расчет средневзвешенной ставки для закрытых депозитов
-----------------------------
ClosedWeightedRates AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         CASE WHEN SUM(BALANCE_RUB) > 0 THEN SUM(BALANCE_RUB * ConvertedRate) / SUM(BALANCE_RUB)
              ELSE 0 END AS WeightedRate_Closed
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
)
-----------------------------
-- Финальный SELECT. Объединяем агрегаты по открытым и закрытым депозитам.
-----------------------------
SELECT
    COALESCE(o.MonthEnd, ow.MonthEnd, c.MonthEnd, cw.MonthEnd) AS MonthEnd,
    ISNULL(COALESCE(o.SegmentGrouping, ow.SegmentGrouping, c.SegmentGrouping, cw.SegmentGrouping), N'Все сегменты') AS SegmentGrouping,
    ISNULL(COALESCE(o.CurrencyGrouping, ow.CurrencyGrouping, c.CurrencyGrouping, cw.CurrencyGrouping), N'Все валюты') AS CurrencyGrouping,
    ISNULL(COALESCE(o.TermBucketGrouping, ow.TermBucketGrouping, c.TermBucketGrouping, cw.TermBucketGrouping), N'Все бакеты') AS TermBucketGrouping,
    ISNULL(o.OpenedDeals,0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub,0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong,0) AS Count_Prolong,
    ISNULL(o.Count_1yProlong,0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong,0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong,0) AS Count_3plusProlong,
    ISNULL(o.Sum_ProlongRub,0) AS Sum_ProlongRub,
    ISNULL(o.Sum_1yProlong_Rub,0) AS Sum_1yProlong_Rub,
    ISNULL(o.Sum_2yProlong_Rub,0) AS Sum_2yProlong_Rub,
    ISNULL(o.Sum_3plusProlong_Rub,0) AS Sum_3plusProlong_Rub,
    -- Средневзвешенные ставки по открытым депозитам:
    ISNULL(ow.WeightedRate_All,0) AS WeightedRate_All,
    ISNULL(ow.WeightedRate_NewNoProlong,0) AS WeightedRate_NewNoProlong,
    ISNULL(ow.WeightedRate_1y,0) AS WeightedRate_1y,
    ISNULL(ow.WeightedRate_2y,0) AS WeightedRate_2y,
    ISNULL(ow.WeightedRate_3plus,0) AS WeightedRate_3plus,
    ISNULL(ow.WeightedRate_Previous,0) AS WeightedRate_Previous,
    -- Показатели по закрытым депозитам:
    ISNULL(c.ClosedDeals,0) AS ClosedDeals,
    ISNULL(c.Summ_ClosedBalanceRub,0) AS Summ_ClosedBalanceRub,
    ISNULL(cw.WeightedRate_Closed,0) AS WeightedRate_Closed,
    CASE WHEN ISNULL(o.Summ_BalanceRub,0) > 0 THEN
         1.0 * c.Summ_ClosedBalanceRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN OpenWeightedRates ow
    ON o.MonthEnd = ow.MonthEnd
   AND o.SegmentGrouping = ow.SegmentGrouping
   AND o.CurrencyGrouping = ow.CurrencyGrouping
   AND o.TermBucketGrouping = ow.TermBucketGrouping
FULL OUTER JOIN ClosedAggregated c
    ON COALESCE(o.MonthEnd, ow.MonthEnd) = c.MonthEnd
   AND COALESCE(o.SegmentGrouping, ow.SegmentGrouping) = c.SegmentGrouping
   AND COALESCE(o.CurrencyGrouping, ow.CurrencyGrouping) = c.CurrencyGrouping
   AND COALESCE(o.TermBucketGrouping, ow.TermBucketGrouping) = c.TermBucketGrouping
FULL OUTER JOIN ClosedWeightedRates cw
    ON c.MonthEnd = cw.MonthEnd
   AND c.SegmentGrouping = cw.SegmentGrouping
   AND c.CurrencyGrouping = cw.CurrencyGrouping
   AND c.TermBucketGrouping = cw.TermBucketGrouping
ORDER BY
    COALESCE(o.MonthEnd, ow.MonthEnd, c.MonthEnd, cw.MonthEnd);
GO
```

────────────────────────────
**Пояснения:**

1. **Часть A (Открытые депозиты):**
   - CTE DealsInMonthRates – выбирает депозиты по DT_OPEN, вычисляет MATUR как разницу между DT_CLOSE_PLAN и DT_OPEN и рассчитывает ConvertedRate через функцию fnc_IntRate.
   - CTE DealsInMonthBucket – присоединяет бакеты срочности по DaysLived.
   - CTE DealsWithProlong – определяет число пролонгаций (ProlongCount). Если ProlongCount > 0, для первого звена (old1) рассчитывается PrevConvertedRate и берётся PrevBalance.
   - CTE OpenAggregated – агрегирует данные по открытым депозитам, включая:
       • Общие показатели: OpenedDeals, Summ_BalanceRub.
       • Показатели по пролонгациям: количества и суммы для 1-й, 2-й и 3+ пролонгаций, а также для сделок без пролонгации (ProlongCount = 0).
       • Суммарное взвешивание ставки: Sum_RateWeighted.
       • Суммарные агрегаты для предыдущих депозитов (для пролонгированных).
   - CTE OpenWeightedRates – по тому же разбиению вычисляет средневзвешенные ставки:
       • WeightedRate_All – для всех открытых.
       • WeightedRate_NewNoProlong – для сделок с ProlongCount = 0 («Новые без пролонгации»).
       • WeightedRate_1y, WeightedRate_2y, WeightedRate_3plus – для соответствующих групп.
       • WeightedRate_Previous – для предыдущих депозитов (до пролонгации).

2. **Часть B (Закрытые депозиты – Выходы):**
   - CTE ClosedDealsInMonthRates – выбирает депозиты по DT_CLOSE_FACT, рассчитывает MATUR и ConvertedRate для закрытых.
   - CTE ClosedDealsInMonthBucket – присоединяет бакеты срочности для закрытых депозитов.
   - CTE ClosedAggregated – агрегирует количество и сумму закрытых депозитов.
   - CTE ClosedWeightedRates – вычисляет средневзвешенную ставку для закрытых депозитов.

3. **Итоговое объединение:**
   - Итоговый SELECT объединяет агрегаты по открытым и закрытым депозитам через FULL OUTER JOIN по четырём ключам.
   - Дополнительно рассчитывается доля выходов ([Доля_выходов_Rub]) как отношение суммы закрытых депозитов к сумме открытых.
   
Этот код позволяет получить детальную сегментацию с итоговыми строками (например, «Все сегменты», «Все валюты», «Все бакеты» и комбинации, например, «Все сегменты, конкретная валюта, бакет '3 месяца'»), а также отдельно смотреть показатели по:
   • Новым сделкам без пролонгации (WeightedRate_NewNoProlong),
   • Пролонгированным сделкам (с разбиением по 1-й, 2-й, 3+ пролонгации),
   • Ставке предыдущих депозитов до пролонгации (WeightedRate_Previous),
   • А также для закрытых депозитов (WeightedRate_Closed).

Все агрегаты рассчитываются с использованием BALANCE_RUB и преобразованных ставок, что позволяет проводить качественный сравнительный анализ.
  
────────────────────────────
Этот код – максимально подробное объединённое решение согласно вашим требованиям.